# **Google trends by Day**

In [69]:
import os
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import time
import random
import pyautogui
import pyperclip
import win32gui
import win32con

data_path = '../data/'

In [70]:
# Parameters
start_date = datetime(2020, 1, 12) #YYYY-MM-DD
end_date = datetime(2023, 8, 31) #YYYY-MM-DD
months_interval = 6
geo_location = 'CO-DC' # CO
search_query = 'Policía'

In [71]:
def generate_date_ranges(start_date, end_date, months_interval):
    """
    Generate date ranges with a specified interval.

    Parameters:
    - start_date (datetime): The start date.
    - end_date (datetime): The end date.
    - months_interval (int): The interval in months.

    Returns:
    - list: A list of tuples representing start and end dates for each interval.
    """

    # Define the interval
    interval = relativedelta(months=months_interval)

    # Initialize the list to store dates
    dates = []

    # Iterate over the date range
    current_date = start_date
    while current_date <= end_date:
        # Calculate the end date by adding the specified months interval
        end_of_interval = current_date + interval

        # Adjust the end date to the last day of the previous month
        end_of_interval -= timedelta(days=1)

        if end_of_interval > end_date:
            dates.append(current_date.strftime("%Y-%m-%d") +" " +end_date.strftime("%Y-%m-%d"))
        else:
            # Append the result to the list
            dates.append(current_date.strftime("%Y-%m-%d") +" " +end_of_interval.strftime("%Y-%m-%d"))

        # Move to the next iteration
        current_date = end_of_interval + timedelta(days=1)
    
    dates.append(start_date.strftime("%Y-%m-%d") +" " + end_date.strftime("%Y-%m-%d"))

    return dates

dates = generate_date_ranges(start_date, end_date, months_interval)
dates

['2020-01-12 2020-07-11',
 '2020-07-12 2021-01-11',
 '2021-01-12 2021-07-11',
 '2021-07-12 2022-01-11',
 '2022-01-12 2022-07-11',
 '2022-07-12 2023-01-11',
 '2023-01-12 2023-07-11',
 '2023-07-12 2023-08-31',
 '2020-01-12 2023-08-31']

In [72]:
# Assuming you have the following variables
urls = []
# Display the result
for i in dates:
    start_date = i.split(" ")[0]
    end_date = i.split(" ")[1][2:]

    url = f'https://trends.google.es/trends/explore?date={start_date}%2020{end_date}&geo={geo_location}&q={search_query}&hl=es'
    urls.append(url)

## **PyAutoGUI - Robot**


In [73]:
def windowEnumHandler(hwnd, top_windows):
    top_windows.append((hwnd, win32gui.GetWindowText(hwnd)))

def bringToFront(window_name):
    top_windows = []
    win32gui.EnumWindows(windowEnumHandler, top_windows)
    for i in top_windows:
        # print(i[1])
        if window_name.lower() in i[1].lower():
            # print("found", window_name)
            win32gui.ShowWindow(i[0], win32con.SW_SHOWMAXIMIZED)
            win32gui.SetForegroundWindow(i[0])
            break

def getData():
    try:
        for url in urls:
            pyautogui.moveTo(300, 75, duration=1) # Position of search bar - Juan José (300, 75)
            pyautogui.tripleClick()
            time.sleep(random.uniform(0.5, 1))
            pyautogui.press('delete')
            time.sleep(random.uniform(0.5, 1))
            pyperclip.copy(url)
            pyautogui.hotkey("ctrl", "v")
            time.sleep(random.uniform(0.5, 1))
            pyautogui.press('enter')
            time.sleep(random.uniform(5, 7))
            pyautogui.moveTo(1610, 710, duration=1) # Position for downloading - Juan José (1610, 710)
            time.sleep(random.uniform(0.5, 1))
            pyautogui.click()
            time.sleep(random.uniform(10, 12)) # Sleep to avoid McCafee Error.
    except KeyboardInterrupt:
        print("Script terminated by user.")

# Run
if __name__ == "__main__":
    window_name = "Google Chrome"
    bringToFront(window_name)
    getData()

## **Final processing**

In [74]:
import os
import pandas as pd

def get_download_folder():
    # Use the USERPROFILE environment variable on Windows
    user_profile = os.environ.get('USERPROFILE')

    # Check if USERPROFILE is set and points to a directory
    if user_profile and os.path.isdir(user_profile):
        # Construct the path to the Downloads folder
        download_folder = os.path.join(user_profile, 'Downloads')
        return download_folder
    else:
        # Default to the current working directory if environment variables are not set or invalid
        return os.getcwd()

# Get the default download folder path
default_download_folder = get_download_folder()

# Get a list of all files in the download folder
files = os.listdir(default_download_folder)

In [75]:
def get_all_trends(csv_files):
    # Weekly data
    weekly_path = os.path.join(default_download_folder, csv_files.pop(-1)) # Remove the last element from the list
    weekly_data = pd.read_csv(weekly_path, encoding='utf-8', skiprows=3, names=['date', 'trends'])
    os.remove(weekly_path)

    # Daily data
    # Read each CSV file and store it in a dictionary
    daily_data = {}
    for csv_file in csv_files:
        file_path = os.path.join(default_download_folder, csv_file)
        file_name = csv_file.split('.')[0] # Eliminate the extension
        daily_data[file_name] = pd.read_csv(file_path, encoding='utf-8', skiprows=3, names=['date', 'trends'])
        os.remove(file_path)         
    # Concatenate all DataFrames in the dictionary into a single DataFrame
    daily_trends = pd.concat(daily_data.values(), ignore_index=True)

    # Sort the DataFrame by the 'date' column
    daily_trends.sort_values(by='date', inplace=True)

    return (daily_trends, weekly_data)

In [76]:
# Filter files that start with 'multiTimeline' and have '.csv' extension
csv_files = [file for file in files if file.startswith('multiTimeline') and file.endswith('.csv')]
csv_files.sort(key=lambda x: os.path.getmtime(os.path.join(default_download_folder, x)), reverse=False)

daily_data, weekly_data = get_all_trends(csv_files)

daily_data.to_csv(os.path.join(data_path, 'final_trends', 'daily', f'{search_query}_daily.csv'), index=False)
weekly_data.to_csv(os.path.join(data_path, 'final_trends', 'weekly', f'{search_query}_weekly.csv'), index=False)

In [77]:
def adjust_trends(df1, df2):
    col1 = 'trends_1'
    col2 = 'trends_2'

    # Merge dataframes
    result_df = pd.merge(df2, df1, how='left', on='date', suffixes=('_2', '_1'))

    # Forward fill the values in the 'Police_1' column
    result_df[col1] = result_df[col1].ffill()

    # result_df[col1] - Diario
    # result_df[col2] - Semanal

    result_df['trends'] = result_df[col1] * result_df[col2] /100

    result_df.drop([col1, col2], axis=1, inplace=True)
    return result_df

In [78]:
final_data = adjust_trends(weekly_data, daily_data)
final_data = final_data[['date','trends']]
final_data.to_csv(os.path.join(data_path, 'final_trends', f'{search_query}_final.csv'), index=False)

In [79]:
final_data

,date,trends
0,2020-01-12,13.65
1,2020-01-13,19.50
2,2020-01-14,12.09
3,2020-01-15,14.82
4,2020-01-16,18.33
...,...,...
1323,2023-08-27,32.24
1324,2023-08-28,42.78
1325,2023-08-29,34.10
1326,2023-08-30,48.98
